In [2]:
import yfinance as yf
import numpy as np
import scipy.stats as si
import matplotlib.pyplot as plt
import mplfinance as mpf
import plotly.graph_objects as go
from datetime import datetime
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

/Users/I750118/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
class BlackScholesModel:
    def __init__(self, S, K, T, r, sigma):
        self.S = S        # Underlying asset price
        self.K = K        # Option strike price
        self.T = T / 365  # Time to expiration in years
        self.r = r        # Risk-free interest rate
        self.sigma = sigma  # Volatility of the underlying asset

    def d1(self):
        return (np.log(self.S / self.K) + (self.r + 0.5 * self.sigma ** 2) * self.T) / (self.sigma * np.sqrt(self.T))
    
    def d2(self):
        return self.d1() - self.sigma * np.sqrt(self.T)
    
    def call_option_price(self):
        return (self.S * si.norm.cdf(self.d1(), 0.0, 1.0) - self.K * np.exp(-self.r * self.T) * si.norm.cdf(self.d2(), 0.0, 1.0))
    
    def put_option_price(self):
        return (self.K * np.exp(-self.r * self.T) * si.norm.cdf(-self.d2(), 0.0, 1.0) - self.S * si.norm.cdf(-self.d1(), 0.0, 1.0))

In [ ]:
class BinomialOptionsPricing:
    def __init__(self, S, K, T, r, sigma, q, n) -> None:
        self.S = S       # Underlying asset price (current price)
        self.K = K       # Option strike price
        self.T = T / 365 # Time to expiration in years
        self.r = r       # Risk-free interest rate
        self.sigma = sigma # Volatility of the underlying asset
        self.q = q       # Dividend yield
        self.n = n       # Number of time steps / height of the binomial tree

    def call_option_price(self):
        delta = self.T / self.n
        u = np.exp(self.sigma * np.sqrt(delta))
        #TODO

In [45]:
class OptionAnalyzer:
    def __init__(self, token) -> None:
        self.ticker = yf.Ticker(token)
        self.current_price = self.ticker.history(period='1d')['Close'][0]
        self.data = pd.DataFrame()

    def get_options_data(self) -> pd.DataFrame:
        dates = self.ticker.options
        options_data_calls = pd.DataFrame()
        options_data_puts = pd.DataFrame()

        for date in dates:
            opt = self.ticker.option_chain(date)
            opt_data_calls = opt.calls
            opt_data_calls['Expiration'] = date
            opt_data_puts = opt.puts
            opt_data_puts['Expiration'] = date
            options_data_calls = pd.concat([options_data_calls, opt_data_calls])
            options_data_puts = pd.concat([options_data_puts, opt_data_puts])

        return options_data_calls, options_data_puts

    def plot_expiry_date(self, contractType, date) -> None:
        if contractType == 'call':
            self.data = self.get_options_data()[0]
            self.data = self.data[self.data['Expiration'] == date]
        else:
            self.data = self.get_options_data()[1]
            self.data = self.data[self.data['Expiration'] == date]

        plt.figure(figsize=(10, 6))
        plt.plot(self.data['strike'], self.data['ask'], 'o')
        plt.xlabel('Strike')
        plt.ylabel('Premium')
        plt.title('Option Premium vs Strike for ' + contractType + ' options expiring on ' + date)
        plt.show()

    def modal(self, modal, maturity) -> pd.DataFrame:
        calls, puts = self.get_options_data()
        calls = calls[calls['Expiration'] == maturity]
        puts = puts[puts['Expiration'] == maturity]
        calls[f'{modal} Price'] = modal(calls['lastPrice'], calls['strike'], self.days_between(calls['Expiration'], datetime.today()), 0.05, 0.2).call_option_price()
        puts[f'{modal} Price'] = modal(puts['lastPrice'], puts['strike'], self.days_between(puts['Expiration'], datetime.today()), 0.05, 0.2).put_option_price()

        return calls, puts
    
    
    def days_between(d1, d2) -> int:
        d1 = datetime.strptime(d1, "%Y-%m-%d")
        d2 = datetime.strptime(d2, "%Y-%m-%d")
        return abs((d2 - d1).days)